## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-13-01-04-43 +0000,nyt,Gavin Newsom’s Former Chief of Staff Arrested ...,https://www.nytimes.com/2025/11/12/us/newsom-a...
1,2025-11-13-01-03-19 +0000,nyt,Air Traffic Crisis Shows Signs of Easing Amid ...,https://www.nytimes.com/2025/11/12/us/politics...
2,2025-11-13-01-03-00 +0000,wsj,Defense-Tech Founder Faces Charges for Online ...,https://www.wsj.com/us-news/law/defense-tech-f...
3,2025-11-13-01-03-00 +0000,wsj,Democrats released new emails from Jeffrey Eps...,https://www.wsj.com/politics/policy/democrats-...
4,2025-11-13-01-01-01 +0000,nypost,Epstein bragged about his ability to provide R...,https://nypost.com/2025/11/12/us-news/epstein-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2360/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
36,trump,37
30,epstein,22
14,shutdown,18
229,government,15
27,new,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
3,2025-11-13-01-03-00 +0000,wsj,Democrats released new emails from Jeffrey Eps...,https://www.wsj.com/politics/policy/democrats-...,152
242,2025-11-12-13-56-28 +0000,nypost,Furious White House reveals Jeffrey Epstein vi...,https://nypost.com/2025/11/12/us-news/jeffrey-...,130
141,2025-11-12-19-28-42 +0000,nypost,Michael Wolff told Epstein he should try to en...,https://nypost.com/2025/11/12/us-news/michael-...,107
231,2025-11-12-15-06-07 +0000,latimes,"Trump 'knew about the girls,' Jeffrey Epstein ...",https://www.latimes.com/politics/story/2025-11...,96
83,2025-11-12-21-50-30 +0000,nyt,New Epstein Emails Alleged Trump Knew of His C...,https://www.nytimes.com/2025/11/12/us/politics...,88


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
3,152,2025-11-13-01-03-00 +0000,wsj,Democrats released new emails from Jeffrey Eps...,https://www.wsj.com/politics/policy/democrats-...
351,86,2025-11-12-02-00-00 +0000,wsj,As data went dark from the government shutdown...,https://www.wsj.com/economy/central-banking/th...
242,63,2025-11-12-13-56-28 +0000,nypost,Furious White House reveals Jeffrey Epstein vi...,https://nypost.com/2025/11/12/us-news/jeffrey-...
129,56,2025-11-12-20-09-00 +0000,wsj,Atlanta Fed President Raphael Bostic will reti...,https://www.wsj.com/economy/central-banking/at...
322,42,2025-11-12-07-18-55 +0000,nypost,Marriott guests left ‘homeless’ as rental comp...,https://nypost.com/2025/11/12/business/marriot...
187,42,2025-11-12-16-50-12 +0000,nypost,Why Jack Schlossberg can’t count on Kennedy fa...,https://nypost.com/2025/11/12/us-news/why-jack...
88,42,2025-11-12-21-40-32 +0000,nypost,NYC rabbis rip Michael Blake for stoking ‘anti...,https://nypost.com/2025/11/12/us-news/rabbis-c...
245,36,2025-11-12-13-51-00 +0000,nyt,India’s Prime Minister Vows Justice After New ...,https://www.nytimes.com/2025/11/11/world/asia/...
155,36,2025-11-12-18-48-19 +0000,wapo,Turkish military plane falls from sky over Geo...,https://www.washingtonpost.com/world/2025/11/1...
0,34,2025-11-13-01-04-43 +0000,nyt,Gavin Newsom’s Former Chief of Staff Arrested ...,https://www.nytimes.com/2025/11/12/us/newsom-a...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
